# Project - Airline AI Assistant

We make an AI Customer Support assistant for an Airline enriched with custom tools. One tool for getting the ticket prices and the other tool for booking the flight.
We create a Chatbot capable of using one tool in a single response and another capable of using multiple tools in one response.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

history looks like this
[
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hello! How can I help?"},
    {"role": "user", "content": "Tell me a joke"},
    {"role": "assistant", "content": "Why did the chicken cross the road..."}
]

In [4]:
# Chat interface
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


## Tools

With tools, we can write a function, and have the LLM call that function as part of its response.

In [5]:
# Function for ticket prices
# This function simulates a tool that provides ticket prices for different destinations.

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [7]:
# Function for booking flights
# This function simulates a tool that books flights for a given destination.
def book_flight(destination_city, date):
    print(f"Tool book_flight called for {destination_city} on {date}")
    city = destination_city.lower()
    if city in ticket_prices:
        return f"Flight to {destination_city} on {date} booked successfully!"
    else:
        return "Booking failed: Unknown destination."


In [8]:
book_flight("Berlin", "2023-10-01")

Tool book_flight called for Berlin on 2023-10-01


'Flight to Berlin on 2023-10-01 booked successfully!'

In [9]:
# Describe our tool functions:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "book_flight",
    "description": "Book a flight to the destination city on a specific date. Call this whenever a customer wants to book a flight, for example when they say 'Book me a flight to this city on this date'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date of the flight in YYYY-MM-DD format",
            },
        },
        "required": ["destination_city", "date"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, 
         {"type": "function", "function": booking_function}]

## Getting OpenAI to use our Tools

We allow OpenAI "to call our tools".

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tools.

## Handle a single tool call message in a model response

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls": #stop response and call tool
        message = response.choices[0].message #collect request to run the tool
        if message.tool_calls[0].function.name == "get_ticket_price":
            response, city = handle_tool_call_price(message)
        elif message.tool_calls[0].function.name == "book_flight":
            response, city = handle_tool_call_booking(message) #collect response from tool
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages) #send back to LLM
    
    return response.choices[0].message.content

In [12]:
# We have to write that function handle_tool_call:
# The function takes a single tool call message in a model response, not muliple tool calls.
# This function unpacks the message from the LLM, extracts the tool available, and executes the function with the provided arguments.

def handle_tool_call_price(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id #links the response to the tool call
    }
    return response, city

def handle_tool_call_booking(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    date = arguments.get('date')
    booking_response = book_flight(city, date)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "date": date, "booking_response": booking_response}),
        "tool_call_id": tool_call.id #links the response to the tool call
    }
    return response, city


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7894
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
Tool book_flight called for Berlin on 2026-01-01


## Handle multiple tool calls message in a single response

In [14]:
def chat_multi(message, history):
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )
    assistant_msg = response.choices[0].message

    # Respond to ALL tools
    if getattr(assistant_msg, "tool_calls", None):
        tool_messages = [dispatch_tool_call(tool_call) for tool_call in assistant_msg.tool_calls]

        # Append the assistant message (with tool_calls) and all tool messages
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content or "",
            "tool_calls": assistant_msg.tool_calls
        })
        messages.extend(tool_messages)

        # Continue the conversation so the model can use tool outputs
        response2 = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools  # keep tools available in case the model chains more calls
        )
        return response2.choices[0].message.content

    # No tool calls—just return the assistant’s content
    return assistant_msg.content

In [15]:
def dispatch_tool_call(tool_call):
    name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments or "{}")

    if name == "get_ticket_price":
        city = arguments.get("destination_city")
        price = get_ticket_price(city)
        tool_response = {"destination_city": city, "price": price}

    elif name == "book_flight":
        city = arguments.get("destination_city")
        date = arguments.get("date")
        booking_response = book_flight(city, date)
        tool_response = {"destination_city": city, "date": date, "booking_response": booking_response}

    else:
        tool_response = {"error": f"Unknown tool '{name}'", "arguments": arguments}

    # Tool message linked by tool_call_id
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(tool_response)
    }



In [16]:
gr.ChatInterface(fn=chat_multi, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
Tool book_flight called for Berlin on 2026-01-01
